In [1]:

import os, sys, math, time
import numpy as np
import numpy.linalg as la
import plotly.graph_objects as go
import plotly.express as ex
from plotly.subplots import make_subplots
import pandas as pd

import json as js
import _pickle as pickle
import bz2
import ray

import torch
import torch.nn as nn
import torchvision
from torch.utils.data import Dataset, TensorDataset
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
from collections import OrderedDict

sys.path.append("../")
import func

In [2]:
torch.cuda.device_count()

1

## Analyse features

In [9]:
data_path = "../../data/"
# load data
data = func.load(data_path+"LOCO_R2-default-locomotion.pbz2")
data_2 = func.load(data_path+"LOCO_R2-default-locomotion-small.pbz2")

modelPath = "/home/nuoc/Documents/MEX/models/MLP4_withLabel_best/0.00324857.512.pbz2"


In [7]:
class MLP(nn.Module):
    def __init__(self, dimensions:list, act_fn, ):

"""
from sebastianstarke {https://github.com/sebastianstarke/AI4Animation/blob/master/AI4Animation/SIGGRAPH_Asia_2019/TensorFlow/NSM/}
and from fabiozinno {https://github.com/electronicarts/character-motion-vaes/blob/main/vae_motion/models.py}
"""

import torch.nn.functional as F
class MoE(nn.Module):
    def __init__(self, dimensions:list, act_fn:str, k_experts:int, gate_size:int, keep_prob:float=0):
        super().__init__()

        self.dimensions = dimensions
        self.act_fn = act_fn
        self.k_experts = k_experts
        self.keep_prob = keep_prob

        self.layers = []

        self.build()
        self.init_params()

        self.gate = nn.Sequential(
            nn.Linear(dimensions[0], gate_size),
            nn.ELU(),
            nn.Linear(gate_size, gate_size),
            nn.ELU(),
            nn.Linear(gate_size, k_experts)
        )

    def forward(self, x:torch.Tensor) -> torch.Tensor:
        coefficients = F.softmax(self.gate(x), dim=1)

        layer_out = x
        for (weight, bias, activation) in self.decoder_layers:
            if weight is None:
                layer_out = activation(layer_out)
            else:
                flat_weight = weight.flatten(start_dim=1, end_dim=2)
                mixed_weight = torch.matmul(coefficients, flat_weight).view(
                    coefficients.shape[0], *weight.shape[1:3]
                )

                input = layer_out.unsqueeze(1)
                mixed_bias = torch.matmul(coefficients, bias).unsqueeze(1)
                out = torch.baddbmm(mixed_bias, input, mixed_weight).squeeze(1)
                layer_out = activation(out) if activation is not None else out

        return layer_out

    def build(self):
        layers = []
        for i, size in enumerate(zip(self.dimensions[0:], self.dimensions[1:])):
            if i < len(self.dimensions) - 2:
                layers.append(
                    (
                        nn.Parameter(torch.empty(self.k_experts, size[0], size[1])),
                        nn.Parameter(torch.empty(self.k_experts, size[1])),
                        self.activation()
                    )
                )
            else:
                layers.append(
                    (
                        nn.Parameter(torch.empty(self.k_experts, size[0], size[1])),
                        nn.Parameter(torch.empty(self.k_experts, size[1])),
                        None
                    )
                )

            if self.keep_prob > 0:
                layers.append((None, None, F.dropout))

        self.layers = layers

    def activation(self):
        if self.act_fn == "elu":
            return F.elu
        elif self.act_fn == "relu":
            return F.relu

    def init_params(self):
        for i, (w, b, _) in enumerate(self.layers):
            if w is None:
                continue

            i = str(i)
            torch.nn.init.kaiming_uniform_(w)
            b.data.fill_(0.01)
            self.register_parameter("w" + i, w)
            self.register_parameter("b" + i, b)

In [8]:
class DEC_AE(nn.Module):
    def __init__(self, encoder:nn.Module, decoder:nn.Module):
        super(DEC_AE, self).__init__()
        self.encoder = encoder
        self.decoder = decoder


    def forward(self, x):
        return self.decoder(self.encoder(x))

In [13]:
# Prepare train data
all_data = []
for d in data:
    d = pickle.loads(d)
    pos = []
    for f in d["frames"]:
        p = [jo["pos"] for jo in f]
        pos.append(p)
    all_data.append(pos)

input_data = np.array([np.concatenate([p for p in j]) for pos in all_data for j in pos])
print(input_data.shape)

(1440, 63)


In [16]:
data_ratio = (.7, .15, .15) # training, validation, testing
SEED = 2021
batch_size = 1

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

x_tensor = torch.from_numpy(input_data).float()
y_tensor = torch.from_numpy(input_data).float()

dataset = TensorDataset(x_tensor, y_tensor)
N = len(dataset)

train_ratio = int(data_ratio[0]*N)
val_ratio = int(data_ratio[1] * N)
test_ratio = int(N-train_ratio-val_ratio)
print("Train: ", train_ratio, ", Validation: ", val_ratio, ", Test: ", test_ratio)

train_set, val_set, test_set = random_split(dataset, [train_ratio, val_ratio, test_ratio], generator=torch.Generator().manual_seed(SEED))

train_loader = DataLoader(dataset=train_set, batch_size=batch_size)
val_loader = DataLoader(dataset=val_set, batch_size=batch_size)
test_loader = DataLoader(dataset=test_set, batch_size=batch_size)


cuda
Train:  1007 , Validation:  216 , Test:  217


In [21]:
# Hyper-parameters
input_dim = input_data.shape[1]
output_dim = input_data.shape[1]
latent_dim = 36         # 12 * 3
encoder_layer_sizes = [input_dim, 256, 256, latent_dim]
decoder_layer_sizes = [latent_dim, 256, 256, output_dim]
num_epochs = 100
learning_rate = 0.001
act_fn = "elu"
keep_prob = .2

# model, loss and scheduler
ae = StackedDenoisingAutoEncoder(encoder_layer_sizes, activation=nn.ELU(), final_activation=nn.ELU())
model = DEC_AE(DEC(36, 36, ae.encoder), ae.decoder)

criterion = nn.MSELoss(reduction="mean")
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.1)

print(model)


DEC_AE(
  (encoder): DEC(
    (encoder): Sequential(
      (0): Sequential(
        (linear): Linear(in_features=63, out_features=256, bias=True)
        (activation): ELU(alpha=1.0)
      )
      (1): Sequential(
        (linear): Linear(in_features=256, out_features=256, bias=True)
        (activation): ELU(alpha=1.0)
      )
      (2): Sequential(
        (linear): Linear(in_features=256, out_features=36, bias=True)
      )
    )
    (assignment): ClusterAssignment()
  )
  (decoder): Sequential(
    (0): Sequential(
      (linear): Linear(in_features=36, out_features=256, bias=True)
      (activation): ELU(alpha=1.0)
    )
    (1): Sequential(
      (linear): Linear(in_features=256, out_features=256, bias=True)
      (activation): ELU(alpha=1.0)
    )
    (2): Sequential(
      (linear): Linear(in_features=256, out_features=63, bias=True)
      (activation): ELU(alpha=1.0)
    )
  )
)


In [22]:
total_step = len(train_loader)
i = 0
n_epochs_no_improve = 5

train_loader_len = float(len(train_loader))
val_loader_len = float(len(val_loader))
test_loader_len = float(len(test_loader))

last_avg_training_loss = 0
min_loss = np.inf
epochs_no_improve = 0
best_model_after_epoch = 0

for epoch in range(num_epochs):
    training_loss = 0
    # training
    for inputs, labels in train_loader:
        # inputs = inputs.to(device)
        # outputs = outputs.to(device)

        pred = model(inputs)
        loss = criterion(pred, labels)
        training_loss+=loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

    last_avg_training_loss = training_loss / train_loader_len
    print ('Epoch [{}/{}], Loss: {:.4f}'
        .format(epoch+1, num_epochs, last_avg_training_loss))

    # early stopping
    with torch.no_grad():
        val_loss = 0
        for inputs, labels in val_loader:
            pred_val = model(inputs)
            loss_val = criterion(pred_val, labels)
            val_loss += loss_val.item()

        val_loss /= val_loader_len
        if min_loss > val_loss:
            min_loss = val_loss
            epochs_no_improve = 0
            best_model_after_epoch = epoch

        else:
            epochs_no_improve+=1
            if epochs_no_improve > n_epochs_no_improve:
                print("Early stopping at Epoch: ", epoch)
                print("last training loss: {:2f}".format(last_avg_training_loss))
                print("achieved best validation loss: {:.4f} after at Epoch {}".format(min_loss, best_model_after_epoch))
                break

# Testing
with torch.no_grad():
    test_loss = 0
    for inputs, labels in test_loader:
        pred_test = model(inputs)
        loss_test = criterion(pred_test, labels)
        test_loss += loss_test.item()

    test_loss /= test_loader_len
    print("Test loss: {:.4f}".format(test_loss))


Epoch [1/100], Loss: 0.1884
Epoch [2/100], Loss: 0.1721
Epoch [3/100], Loss: 0.1721
Epoch [4/100], Loss: 0.1721
Epoch [5/100], Loss: 0.1721
Epoch [6/100], Loss: 0.1721
Epoch [7/100], Loss: 0.1721
Early stopping at Epoch:  6
last training loss: 0.172100
achieved best validation loss: 0.1820 after at Epoch 0
Test loss: 0.1849
